In [6]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader #UnstructuredPDFLoader

from packages.functions import set_env_variables, instantiate_llm

set_env_variables()

It's all right!


In [7]:
llm = instantiate_llm(model_name="gpt-3.5-turbo")

In [8]:
loader = PyPDFLoader('./files/online_p_value_calculator.pdf')
data = loader.load()

# llm.get_num_tokens(data)

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=12000, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

len(chunks)

10

In [10]:
from packages.functions import print_embedding_cost

print_embedding_cost(chunks)

Total Tokens: 927
Embedding Cost in USD: 0.000371


In [11]:
chain = load_summarize_chain(
    llm,
    chain_type="refine",
    verbose=False
)

output = chain.run(chunks)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-va3LEBvnLYt8QWkUrWq6m1TC on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-va3LEBvnLYt8QWkUrWq6m1TC on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.

In [ ]:
print(output)

## customize Prompts

In [ ]:
prompt_template = """
    Write a concise summary of the following extracting the key information:
    Text: `{text}`
    CONCISE SUMMARY:
"""
initial_prompt = PromptTemplate(template=prompt_template, input_variables=["text"])


refine_template = """
    Your job is to provide a final summary.
    I have provided an existing summary up to a certain point. {existing_answer}.
    Please refine the existing summary with some more context below
    -----------
    {text}
    -----------
    Start the final summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWING by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
"""
refine_prompt = PromptTemplate(template=refine_template, input_variables=["existing_answer", "text"])

In [ ]:
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=initial_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=False
)
output = chain(chunks)

In [ ]:
print(output)